### What is the Light GBM?



Trước hết để tìm hiểu rõ hơn vè Light GBM chúng ta cần biết về các thuật toán Tree Base algorithm as decision trees, random forest, gradient boosting. Trong bài viết này nó sẽ tương tối dài tôi chia làm 3 phần 

* Phần 1: Các thuật toán Tree Base Algorithms (giải thích lí thuyết)

* Phần 2: Light GBM

* Phần 3: How to fine turn parameter of Light GBM.



### Phần 1: Các thuật toán Tree Base Algorithms

Nhìn tổng quan case study phần 1 chúng ta cần làm rõ các vấn đề sau:

1.1  What is a Decision Tree? How does it work?

1.2  Regression Trees vs Classification Trees

1.3  How does a tree based algorithms decide where to split?

1.4  What are the key parameters of model building and how can we avoid over-fitting in tree based algorithms?

1.5  Are tree based algorithms better than linear models?

1.6  Working with Decision Trees in R and Python

1.7  What are the ensemble methods of tree based algorithms?

1.8  What is Bagging? How does it work?

1.9  What is Random Forest ? How does it work?

1.10 What is Boosting ? How does it work?

1.11 Which is more powerful: GBM or Xgboost?

1.11 Working with GBM in R and Python

1.12 Working with Xgboost in R and Python


Trong tài liệu này sử dụng demo code bằng ngôn ngữ lập trình Python nên các vấn đề thực hiện vs R sẽ được bỏ qua.

### 1.1  + 1.2 Cây quyết định

Cây quyết định là gì và nó hoạt động ra sao?

a. Tổng quan

- Cây quyết định là một loại thuật toán học tập có giám sát được sử dụng nhiều trong bài toán phân loại.

- Có 2 loại cây quyết định phụ thuộc vào mục tiếp áp dụng nó: mục tiêu phân loại với mục tiêu liêu tục ở biến đầu ra.

Vậy cây quyết định hoạt động ra sao cùng xem ví dụ sau:

Xét bài toán: Có 30 học sinh gồm 3 biến đặc trưng cơ bản:

x: giới tính (Nam/Nữ)
y: lớp (IX, X)
z: chiều cao ( 5-6 ft)

15/30 chơi mobile ledgend bang bang trong thời gian giải trí. 

Mục đích chúng ta cần phân loại nhóm sinh viên ra 2 loại chơi game MLBB tron giờ giải trí ra khi biết các đắc trưng đầu vào x,y,z với từng học sinh.

Để giải bài toán này bằng cây quyết định ta phải thử lần lượt các thuộc tính:

Xét thuộc tính x =[Nam, Nu]

Với thuộc tính này chia tập dữ liệu thành 2 nhóm:

 * Nhóm 1: Toàn sinh viên Nam (20 sinh viên, 13 người chơi game)  
 * Nhóm 2: Toàn sinh viên Nữ  (10 sinh viên, 2 người chơi game)
 
Tương tự cho 2 loại thuộc tính như mỗi thuộc tính ta đc 2 nhóm

Thuộc tính y: 

* Nhóm 1: Học sinh lớp IX ( 14 người, 6 người chơi game) 
* Nhóm 2: Học sinh lớp X ( 16 người, 9 người chơi game)

Thuộc tính z:

* Nhóm 1: Học sinh cao trên >= 5.5ft  (18 người, 10 người chơi game)
* Nhóm 2: Học sinh thấp hơn < 5.5ft (12 người, 5 người chơi game)

Vậy thì lựa chọn thuộc tính nào để phân loại? Tiêu chí nào đánh giá thuộc tính nào phân loại tốt hơn.

Trước khi khám phá các tiêu chí lựa chọn ta tổng quan qua các định nghĩa, thuật ngữ trên 1 cây:

![](images/tree.jpg)

Nốt gốc: It represents entire population or sample and this further gets divided into two or more homogeneous sets.

Phân chia: It is a process of dividing a node into two or more sub-node

Nốt quyết định sự phân chia : When a sub-node splits into further sub-nodes, then it is called decision node.

Nốt lá: Nodes do not split is called Leaf or Terminal node.

Nhánh: A sub section of entire tree is called branch or sub-tree.

Nốt bố và nốt con: A node, which is divided into sub-nodes is called parent node of sub-nodes where as sub-nodes are the child of parent node.

Các thuật toán tree base algorithms đều dựa trên cấu trúc cây như vậy. Tuy nhiên khác nhau ở chỗ lựa chọn nốt để phân chia và hình thức phân chia sẽ khác nhau.

Trong thuật toán cây quyết định có nhiều thuật toán để xác định nốt quyết định phân chia thông thường gồm có 4 cách sau:

#### Gni ( chỉa số Gni)

- Ap dụng cho phân chia cây dạng nhị phân
- Một tập bất kì khi ta chia 2 tập con từ tập gốc thì xác xuất mỗi tập được chia ra là như nhau và xác xuất khi tập đó đồng nhất ( chỉ có 1 loại ) là 1
- Chỉ số Gni càng cao thì chứng tỏ chỉ số đồng nhất càng cao tức mỗi tập đó gần giống nhau nhiều nhất 

Cách tính chỉ số Gni:

Tính toán Gini cho các nút phụ, sử dụng tổng công thức bình phương xác suất thành công và thất bại (p ^ 2 + q ^ 2).
Tính toán Gini cho phân chia bằng cách sử dụng điểm Gini có trọng số của từng nút của phân chia đó

Ap dung cho ví dụ bên trên:

Split on Gender:

Calculate, Gini for sub-node Female = (0.2)*(0.2)+(0.8)*(0.8)=0.68

Gini for sub-node Male = (0.65)*(0.65)+(0.35)*(0.35)=0.55

Calculate weighted Gini for Split Gender = (10/30)*0.68+(20/30)*0.55 = 0.59

Similar for Split on Class:

Gini for sub-node Class IX = (0.43)*(0.43)+(0.57)*(0.57)=0.51

Gini for sub-node Class X = (0.56)*(0.56)+(0.44)*(0.44)=0.51

Calculate weighted Gini for Split Class = (14/30)*0.51+(16/30)*0.51 = 0.51

#### Chi-Square

Chỉ số này cố gắng thống kê và tìm kiếm sự khác biệt giữa các nút cha và nút phụ( nút mở rộng) bằng tổng bình phương của sự khác biệt được tiêu chuẩn hóa giữa tần số quan sát và tần suất dự kiến của biến mục tiêu.

Gía trị Chi càng cao thấy sự khác biệt giữu cha và nút phụ càng lớn.

Hoạt động với mục tiêu phân loại áp dụng cho sự phân chia không phải là nhị phân

Các bước tính áp dụng cho bài toán trên:

![](images/chi-square.jpg)

#### Information Gain

![](images/gain_des.jpg)

Theo trực giác quan sát được hình sẽ ít tạp chất hơn 2 hình A và B. Do tập C có nhiều các points tương đồng. Từ đó ta suy ra được C cần ít thông tin hơn để mô tả nó . 

Lý thuyết thông tin là một biện pháp để xác định mức độ vô tổ chức này trong một hệ thống được gọi là Entropy. Nếu mẫu hoàn toàn đồng nhất, thì entropy bằng 0 và nếu mẫu được chia đều (50% - 50%), thì nó có entropy là một.

Công thức tính entropy 

#### entropy = -p*$log_2 (p) $ -q * $log_2 (q)$

Steps to calculate entropy for a split:

Calculate entropy of parent node

Calculate entropy of each individual node of split and calculate weighted average of all sub-nodes available in split.
 
Example: Let’s use this method to identify best split for student example.

Entropy for parent node = -(15/30) log2 (15/30) – (15/30) log2 (15/30) = 1. Here 1 shows that it is a impure node.

Entropy for Female node = -(2/10) log2 (2/10) – (8/10) log2 (8/10) = 0.72 and 

for male node,  -(13/20) log2 (13/20) – (7/20) log2 (7/20) = 0.93

Entropy for split Gender = Weighted entropy of sub-nodes = (10/30)*0.72 + (20/30)*0.93 = 0.86

Entropy for Class IX node, -(6/14) log2 (6/14) – (8/14) log2 (8/14) = 0.99

and for Class X node,  -(9/16) log2 (9/16) – (7/16) log2 (7/16) = 0.99.

Entropy for split Class =  (14/30)*0.99 + (16/30)*0.99 = 0.99

ở đây ta thấy entropy của phân chia theo giới tính thấp nhất nên sẽ chơi thuộc tính giới tính là nốt quyết định. Khi entropy thấp thì có nghĩa là việc phân tách có ý nghĩa hơn, bớt lộn xộn hơn có một tập có tính đồng nhất cao hơn.


#### Reduction in Variance (giảm phương sai)

Với 3 cách trên là áp dụng cho mục tiêu bài toán phân loại còn thuật toán này áp dụng cho mục tiêu là bài toán hồi quy (mục tiêu liên tục).

Thuật toán này sử dụng công thức chuẩn của phương sai để chọn phân chia tốt nhất. Sự phân chia với phương sai thấp hơn được chọn làm tiêu chí để phân chia dân số:

Nhắc lại: Phương sai là gì? Phương sai đo độ phân tán dữ liệu, được tính bằng khoảng cách trung bình từ từng số liệu đến số liệu trung bình. 
Phương sai càng nhỏ thì có nghĩa các điểm liệu phân bố càng gần xung quanh điểm trung bình.

Với thuật toán trên người ta chọn thuộc tính và xem xem xùng qunah nó càng nhiều điểm gần nó càng tốt.

Các bước để tính phương sai:

Tính toán phương sai cho mỗi nút.

Tính toán phương sai cho mỗi phân chia là trung bình trọng số của từng phương sai nút.

Ví dụ: - Hãy gán giá trị số 1 cho cricket chơi và 0 cho không chơi cricket. Bây giờ hãy làm theo các bước để xác định phân chia đúng:

Phương sai cho nút Root, ở đây giá trị trung bình là (15 * 1 + 15 * 0) / 30 = 0,5 và chúng ta có 15 một và 15 không. 

Phương sai cho nút gốc là: var_root =  (15 * (1-0.5) ^ 2 + 15 * (0-0.5) ^ 2) / 30 = 0.25

Giá trị trung bình của nút Nữ = (2 * 1 + 8 * 0) /10 = 0.2 và Phương sai = (2 * (1-0.2) ^ 2 + 8 * (0-0.2) ^ 2) / 10 = 0.16

Giá trị trung bình của nút nam =(13 * 1 + 7 * 0)/20=0,65 và phương sai = (13 * (1-0,65) ^ 2 + 7 * (0-0,65) ^ 2) / 20 = 0,23

Phương sai cho phân chia giới tính = Phương sai trọng số của các nút phụ = (10/30) * 0,16 + (20/30) * 0,23 = 0,21

Giá trị trung bình của nút Lớp IX = (6 * 1 + 8 * 0)/14=0.43 và Phương sai =(6 * (1-0.43) ^ 2 + 8 *(0-0.43) ^ 2) / 14 = 0.24

Giá trị trung bình của nút Lớp X = (9 * 1 + 7 * 0) /16=0.56 và Phương sai =(9 *(1-0.56) ^ 2 + 7 * (0-0.56) ^ 2) / 16 = 0.25

Phương sai cho phân chia giới tính = (14/30) * 0,24 + (16/30) * 0,25 = 0,25

Ở trên, bạn có thể thấy rằng Phân chia giới tính có phương sai thấp hơn so với nút cha, do đó việc phân chia sẽ diễn ra trên biến Giới tính .

#### Phân loại mục tiêu của thuật toán Tree Base Algorithm: Regression Trees vs Classification Trees

Có 2 loại chính là: Regression Trees vs Classification Trees (bài toán hồi quy và bài toán phân loại)


* Cây hồi quy được sử dụng khi biến phụ thuộc là liên tục. Cây phân loại được sử dụng khi biến phụ thuộc là phân loại.
* Cả hai cây theo một cách tiếp cận tham lam từ trên xuống được gọi là chia nhị phân đệ quy. Chúng tôi gọi nó là 'từ trên xuống' bởi vì nó bắt đầu từ ngọn cây khi tất cả các quan sát có sẵn trong một khu vực và liên tiếp chia không gian dự đoán thành hai nhánh mới xuống cây. Nó được gọi là 'tham lam' bởi vì, thuật toán quan tâm (tìm kiếm biến tốt nhất có sẵn) chỉ về sự phân chia hiện tại, và không phải về sự phân chia trong tương lai sẽ dẫn đến một cây tốt hơn.
* Quá trình phân tách này được tiếp tục cho đến khi đạt được tiêu chí dừng do người dùng xác định. Ví dụ: chúng ta có thể yêu cầu thuật toán dừng lại khi số lượng quan sát trên mỗi nút trở nên ít hơn 50.
* Trong cả hai trường hợp, quá trình phân tách dẫn đến cây được trồng hoàn toàn cho đến khi đạt được tiêu chí dừng. Nhưng, cây phát triển đầy đủ có khả năng phù hợp với dữ liệu, dẫn đến độ chính xác kém trên dữ liệu không nhìn thấy. Điều này mang lại 'cắt tỉa'. Cắt tỉa là một trong những kỹ thuật được sử dụng giải quyết quá mức

#### Thuật toán cây quyết định: Overfitting

Trên thuật toán cây quyết định nó có thể cho bạn dự đoán chính xác 100% trên tập dữ liệu train nhưng trên tập test nó lại cho kết quả quá tệ. Điều này là việc khi bạn để kích thước cây quá lớn mỗi lần phân chia nó sẽ lấy 1 lá. 

Để khác phục sự quá mức trên tập dữ liệu train này chúng ta có 2 cách xử lí:

+ Kiểm soát kích thước của cây

+ Căt tỉa cây

Với phương pháp đầu tiên các thông số cần kiểm soát là:

![](images/ovf.jpg)

* Mẫu tối thiểu để phân chia nút trong hình trên là dưới 30 mẫu sẽ không được phân chia nốt

* Mẫu tối thiểu cho một nút lá

* Độ sâu của cây 

* Số nút lá tối đa

* Các tính năng tối đa để phân chia( số lượng thuộc tính dùng để phân chia)

Với phương pháp cắt tỉa cây?

![](images/cut.jpg)



Trong cây quyết định nó chỉ xác định mục tiêu tốt nhất trong TH hiện tại và không quan tâm trạng thái sau đó. Cũng tương tự như trường hợp bên trên hình vẽ. Bạn có 2 quyết định:
Bạn sẽ rẽ sang trái vượt qua 2 chiếc xe bên trên bạn, và tiến sát đến xe tải và di chuyển với vận tốc 30km/h. Mục tiêu bạn được tối đa hóa trong khoảng thời gian ngắn. Nhưng sau đó khi sát xe tải bạn sẽ bị bỏ lại bởi đoàn xe bên phải. Chính vì thế đó là điểm yếu của thuật toán cây quyết định. 

Cách thức thực hiện cắt tỉa cây quyết định:

Bước 1: Làm cho cây quyết định đến một độ sâu lớn(nhất định).

Bước 2: Đi từ phần lá đi lên loại bỏ lá cho lợi nhuận âm.

Bước 3: Giả sử một sự phân chia mang lại cho chúng ta mức tăng -10 (mất 10) và sau đó sự phân chia tiếp theo mang lại cho chúng ta mức tăng 20. Một cây quyết định đơn giản sẽ dừng lại ở bước 1 nhưng khi cắt tỉa, chúng ta sẽ thấy rằng tổng thể đạt được +10 và giữ cả hai lá.

#### Các thuật toán tree base algorithms có tốt hơn các mô hình tuyến tính hay không?

Đó là việc xem xét bài toán của bạn theo các tiêu chí sau:

* Nếu mối quan hệ giữa biến phụ thuộc & biến độc lập được xấp xỉ bằng mô hình tuyến tính, hồi quy tuyến tính sẽ vượt trội hơn mô hình dựa trên cây.
* Nếu có mối quan hệ phi tuyến tính & phức tạp cao giữa các biến phụ thuộc và độc lập, mô hình cây sẽ vượt trội hơn phương pháp hồi quy cổ điển.
* Nếu bạn cần xây dựng một mô hình dễ giải thích cho mọi người, mô hình cây quyết định sẽ luôn làm tốt hơn mô hình tuyến tính. Các mô hình cây quyết định thậm chí còn đơn giản để giải thích hơn hồi quy tuyến tính!

In [ ]:
#Import Library
#Import other necessary libraries like pandas, numpy...
from sklearn import tree
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create tree object 
model = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score
model.fit(X,y)
model.score(X, y)
#Predict Output
predicted= model.predict(x_test)

#### Ensemble methods in Tree Based Algorithms

Nếu mô hình của chúng ta quá đơn giản và có rất ít tham số thì nó có thể có độ lệch cao và phương sai thấp. Mặt khác, nếu mô hình của chúng tôi có số lượng lớn các tham số thì nó sẽ có phương sai cao và độ lệch thấp. Vì vậy, chúng ta cần tìm sự cân bằng đúng / tốt mà không cần quá mức và thiếu dữ liệu.

![](images/1_RQ6ICt_FBSx6mkAsGVwx8g.png)

Khi xây dựng mô hình cần phải cân bằng giữa bias và phương sai, có một số pp cải thiện điều này là: Bagging, Boosting and Stacking.

#### Thuật toán triển khai phương pháp Bagging: Random Forest
    
Rừng ngẫu nhiên là một trong thuật toán dùng được trong mọi tình huống nếu chữa nghĩ ra phương pháp nào đối với tất data science problems. Nó giống việc không biết chơi con gì trong Bang Bang thì mình sẽ pick Selena haha. Khá là trùng hợp à nha à hihi.

Random Forest là một phương pháp học máy đa năng có khả năng thực hiện cả các nhiệm vụ hồi quy và phân loại. Nó cũng thực hiện các phương pháp giảm kích thước, xử lý các giá trị còn thiếu, giá trị ngoại lệ và các bước thiết yếu khác của khám phá dữ liệu và thực hiện công việc khá tốt. Nó là một loại phương pháp học tập đồng bộ , trong đó một nhóm các mô hình yếu kết hợp với nhau để tạo thành một mô hình mạnh mẽ.

Nó hoạt động ra sao:

Ys tưởng:
Đối với thuật toán cây quyết định chúng ta chỉ dùng một cây duy nhất cho tác vụ phân loại hoặc hồi quy, trong rừng ngẫu nhiên chúng ta trồng nhiều cây một cách ngẫu nhiên. Mỗi cây đưa ra một phân loại và nói rằng cây đã đưa ra 1 phiếu bầu cho lớp đó và tổng số phiếu bầu lớp nào cao nhất là đầu ra dự đoán. Công nhận cũng hay.


Vậy trồng cây thế nào?

1. Gỉa sử có n mẫu dữ liệu cho huấn luyện, ta lấy ngẫu nhiên tập con trong số n mẫu huấn luyện đã cho và có thay thế mẫu con trong quá trình huấn luyện.
2. Gỉa sử có M giá trị đầu vào, ta sử dụng m giá trị ( m < M) để xây dựng cây như cây quyết định
3. Mỗi cây được trồng kích thước lớn nhất và không cắt tỉa.
4. Dự đoán là tổng hợp dữ đoán của tất cả các cây. 

Ưu điểm của rừng ngẫu nhiên?

* Thuật toán này giải quyết được cả vấn đề hồi quy và phân loại.

* Một trong những lợi ích của Rừng ngẫu nhiên làm tôi phấn khích nhất là, sức mạnh xử lý tập dữ liệu lớn với tính chiều cao hơn. Nó có thể xử lý hàng ngàn biến đầu vào và xác định hầu hết các biến quan trọng để nó được coi là một trong những phương pháp giảm kích thước.

* Nó có một phương pháp hiệu quả để ước tính dữ liệu bị thiếu và duy trì độ chính xác khi thiếu một tỷ lệ lớn dữ liệu.
* Nó có các phương thức để cân bằng các lỗi trong các tập dữ liệu trong đó các lớp bị mất cân bằng.
Các khả năng của ở trên có thể được mở rộng thành dữ liệu chưa được gắn nhãn, dẫn đến phân cụm không được giám sát, xem dữ liệu và phát hiện ngoại lệ.
* Random Forest bao gồm lấy mẫu dữ liệu đầu vào với sự thay thế được gọi là lấy mẫu bootstrap. Ở đây một phần ba dữ liệu không được sử dụng cho đào tạo và có thể được sử dụng để kiểm tra. Chúng được gọi là mẫu ra khỏi túi . Lỗi ước tính trên các mẫu trong túi này được gọi là  lỗi túi . Nghiên cứu ước tính lỗi bằng Out of bag, đưa ra bằng chứng cho thấy ước tính hết túi là chính xác như sử dụng một bộ kiểm tra có cùng kích thước với tập huấn luyện. Do đó, việc sử dụng ước tính lỗi xuất túi sẽ loại bỏ sự cần thiết của một bộ kiểm tra dành riêng.

Nhược điểm:
* Rất tốt cho phân loại, không tốt cho hồi quy.

In [ ]:
'''
The following code is for the Random Forest
Created by - ANALYTICS VIDHYA
'''

# importing required libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# read the train and test dataset
train_data = pd.read_csv('train-data.csv')
test_data = pd.read_csv('test-data.csv')

# view the top 3 rows of the dataset
print(train_data.head(3))

# shape of the dataset
print('\nShape of training data :',train_data.shape)
print('\nShape of testing data :',test_data.shape)

# Now, we need to predict the missing target variable in the test data
# target variable - Survived

# seperate the independent and target variable on training data
train_x = train_data.drop(columns=['Survived'],axis=1)
train_y = train_data['Survived']

# seperate the independent and target variable on testing data
test_x = test_data.drop(columns=['Survived'],axis=1)
test_y = test_data['Survived']

'''

Create the object of the Random Forest model
You can also add other parameters and test your code here
Some parameters are : n_estimators and max_depth
Documentation of sklearn RandomForestClassifier: 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

'''
model = RandomForestClassifier()

# fit the model with the training data
model.fit(train_x,train_y)

# number of trees used
print('Number of Trees used : ', model.n_estimators)

# predict the target on the train dataset
predict_train = model.predict(train_x)
print('\nTarget on train data',predict_train) 

# Accuray Score on train dataset
accuracy_train = accuracy_score(train_y,predict_train)
print('\naccuracy_score on train dataset : ', accuracy_train)

# predict the target on the test dataset
predict_test = model.predict(test_x)
print('\nTarget on test data',predict_test) 

# Accuracy Score on test dataset
accuracy_test = accuracy_score(test_y,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)


#### Phương pháp cân bằng phương sai và độ lệch thứ hai là Boosting

Có một bài nói riêng. 